In [1]:
import pandas as pd
import numpy as np
import json
import os
import random
import string
import re
import matplotlib.pyplot as plt
import pickle

from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error
import optuna.integration.lightgbm as lgbo

import umap
pd.set_option('display.max_columns', 2000)

SEED = 777



In [2]:
df_train = pd.read_pickle('../data/train/full_comp.pickle')
df_test = pd.read_pickle('../data/test/full_comp.pickle')
df_train = pd.concat([df_train.drop('cites', axis=1), df_test])
df_train = df_train.reset_index(drop=True)
df_train.shape

(910608, 2001)

In [3]:
df_train.head(1)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,pred_doi_cites,category_main_detail,category_main,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10

In [4]:
drop_cols = []
for col in df_train.columns:
    if '_x' in col or 'doi_cites' in col: #  or col in importance_0_cols
        drop_cols.append(col)
drop_cols.extend(['authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique',
    ])

In [5]:
df_train[df_train['id'].isnull()]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date_x,authors_parsed,doi_cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,pred_doi_cites,category_main_detail,category_main,acc-phys_x,adap-org_x,alg-geom_x,ao-sci_x,astro-ph_x,atom-ph_x,bayes-an_x,chao-dyn_x,chem-ph_x,cmp-lg_x,comp-gas_x,cond-mat_x,cs_x,dg-ga_x,econ_x,eess_x,funct-an_x,gr-qc_x,hep-ex_x,hep-lat_x,hep-ph_x,hep-th_x,math_x,math-ph_x,mtrl-th_x,nlin_x,nucl-ex_x,nucl-th_x,patt-sol_x,physics_x,plasm-ph_x,q-alg_x,q-bio_x,q-fin_x,quant-ph_x,solv-int_x,stat_x,supr-con_x,acc_x,adap_x,alg_x,ao_x,astro_x,atom_x,bayes_x,chao_x,chem_x,cmp_x,comp_x,cond_x,cs_y,dg_x,econ_y,eess_y,funct_x,gr_x,hep_x,math_y,mtrl_x,nlin_y,nucl_x,patt_x,physics_y,plasm_x,q_x,quant_x,solv_x,stat_y,supr_x,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10

In [6]:
# 5foldでtrain全量の予測スコアを振る
NFOLDS = 3
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)

df_train['fold_no'] = [0 for x in range(len(df_train))]

for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold_no'] = fold


lgb_params = {'objective': 'regression',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'lambda_l1': 6.994350738974102e-07,
 'lambda_l2': 0.00019811662033454987,
 'learning_rate': 0.1,
 'feature_fraction': 0.8,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'min_child_samples': 200,
 'num_iterations': 10000,
 'early_stopping_round': 20,
}


result_y = []
result_y_id = []
lgb_result_proba_valid = []

for fold_no in range(NFOLDS):
    target = 'doi_cites'

    valid_fold_no = fold_no

    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([valid_fold_no])][target].values
    y_valid = train[train['fold_no'] == valid_fold_no][target].values

    train = train.drop(drop_cols, axis=1)
    
    x_train = train[~train['fold_no'].isin([valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]

    result_y_id.extend(x_valid['id'].values)
    x_train = x_train.drop(['id', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['id', 'fold_no'], axis=1)

    print(x_train.shape)
    # LightGBM
    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                verbose=100,
    )

    pickle.dump(model, open(f'../models/pred_doi_cites_lgb_{fold_no}.pickle', 'wb'))
    feature = x_train.copy()
    if fold_no == 0:
        df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance_0']).reset_index().rename(columns={'index':'name'}).reset_index()
    else:
        df_temp = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=[f'importance_{fold_no}']).reset_index().rename(columns={'index':'name'}).reset_index().drop('index', axis=1)
        df_feature = pd.merge(df_feature, df_temp, on='name', how='left')

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y.extend(y_valid)
    #lgb_result_proba_test.extend(fold_result_test)
    lgb_result_proba_valid.extend(fold_result_valid)
    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    #print(f"fold {fold_no} lgb score: {rmsle}")


print("valid", mean_squared_error(result_y, lgb_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, lgb_result_proba_test, squared=False))


(607072, 1022)
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=6.994350738974102e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.994350738974102e-07
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.00019811662033454987, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00019811662033454987
[

KeyboardInterrupt: 

In [ ]:
# testも予測スコア振る


In [ ]:
pd.DataFrame(result_y, result_y_id).reset_index().rename(columns={0:'pred_doi_cites', 'index':'id'}).to_pickle('../data/pred_doi_cites.pickle')

In [ ]:
df_train = pd.read_pickle('../data/train/raw.pickle')[['id']]
df_test = pd.read_pickle('../data/test/raw.pickle')[['id']]

In [ ]:
df_feature = pd.read_pickle('../data/pred_doi_cites.pickle')
df_feature.shape

In [ ]:
pd.merge(df_train, df_feature, on='id', how='left').to_pickle('../data/train/pred_doi_cites.pickle')

In [ ]:
pd.merge(df_test, df_feature, on='id', how='left').to_pickle('../data/test/pred_doi_cites.pickle')